In [1]:
import torch
import torchvision
from torch import nn
import torchvision.transforms as transforms
from VGG_Face_torch import VGG_Face_torch
import argparse
import torch.optim as optim
from torch.autograd import Variable
import os
import csv
import cv2
import matplotlib.pyplot as plt
from skimage import io, transform
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from PIL import Image

In [2]:
# parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
batch_size =  16
test_batch_size =  16
epochs = 20
lr = 0.001
momentum = 0.9
seed=1
log_interval = 30
kwargs = {'num_workers': 4, 'pin_memory': True}


In [3]:
EXP_TYPE = 'Classification/'
textMAIN = '/timo/datasets/Dua/GRU-test/Data-Combinations/'
textPATH = textMAIN+EXP_TYPE
COMB = ['BDI','AVEC','AVEC-BDI','AVEC-TEST','BDI-TEST']
NUM = ['1','2','3','4','5']
trainlist=textPATH+COMB[0]+'train'+NUM[0]+'.txt'
testlist=textPATH+COMB[0]+'test'+NUM[0]+'.txt'


In [4]:
# # https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
# # torch.utils.data.Dataset
# class DepressionDataset():
#     """Face Landmarks dataset."""

#     def __init__(self, csv_file, transform=None):
#         """
#         Args:
#             csv_file (string): Path to the csv file with annotations.
#             transform (callable, optional): Optional transform to be applied
#                 on a sample.
#         """
#         with open(csv_file, 'r') as file:
#             self.lines = file.readlines()
#             img_paths=[]
#             labels = []
#             for i in range(len(self.lines)):
#                 info = self.lines[i].rstrip()
#                 par_name = info.split(" ")[0]
#                 imgs = os.listdir(par_name)
# #                 img_path = par_name
#                 imgs.sort()
#                 for img in range(len(imgs)):
#                     img_path = par_name+"/"+imgs[img]
#                     img_paths.append(img_path)
#                     labels.append(int(info.split(" ")[1]))      
# #                     print(img_paths)
#         self.transform = transform
#         self.img_paths = img_paths
#         self.labels = labels

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
# #         print(self.img_paths[128])
# #         idx = len(self.labels)
# #         print(idx)


#         # for i in range(idx):
#         #     info = self.lines[i].rstrip()
#         #     par_name = info.split(" ")[0]
#         #     label = int(info.split(" ")[1])
#         #     imgs = os.listdir(par_name)
#         #     imgs.sort()
#         #     for img in range(len(imgs)):
#         #         img_path = par_name+"/"+imgs[img]
#         img_path = self.img_paths[idx]
#  #       print(img_path)
#         label = self.labels[idx]
#  #       print(label)
#         image = Image.fromarray(io.imread(img_path))
#         sample = {'image': image, 'label': label}
#         # print(sample['label'])
#         if self.transform:
#             transform_img = self.transform(sample['image'])
#             #print(transform_img)
#             sample = {'image': transform_img, 'label': sample['label']}
#             #print(sample['label'])
#         return sample

In [5]:
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
# torch.utils.data.Dataset
class DepressionDataset():
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        with open(csv_file, 'r') as file:
            self.lines = file.readlines()
            img_paths=[]
            labels = []
            for i in range(len(self.lines)):
                info = self.lines[i].rstrip()
                par_name = info.split(" ")[0]
                imgs = os.listdir(par_name) 
                imgs.sort()
                for img in range(len(imgs)):
                    img_path = par_name+"/"+imgs[img]
                    img_paths.append(img_path)
                    labels.append(int(info.split(" ")[1])) 
        self.transform = transform
        self.img_paths = img_paths
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):   
        img_path = self.img_paths[idx] 
        label = self.labels[idx] 
        image = Image.fromarray(io.imread(img_path))
        
        if self.transform:
            image = self.transform(image)  
            
        sample = {'image': image, 'label': label}  
        return sample

In [6]:
transform_train = transforms.Compose([transforms.RandomHorizontalFlip(),
                                 #transforms.ToPILImage(),
                                 transforms.Resize(256),
                                #  transforms.RandomResizedCrop((224,224), scale=(0.875, 1.125), ratio=(1.0, 1.0)),
                                #  transforms.CenterCrop(224),
                                 transforms.RandomCrop(224),
                                 transforms.Resize(224),
                                 transforms.ToTensor(),
                                 #transforms.Grayscale(num_output_channels=1),
                                 transforms.Normalize((0.507395516207, ),(0.255128989415, ))
                                ])

transform_test  = transforms.Compose([transforms.Resize((224,224),interpolation=Image.NEAREST),
                                 #transforms.ToPILImage(),
                                 transforms.ToTensor(),
                                 #transforms.Grayscale(num_output_channels=1),
                                 transforms.Normalize((0.507395516207, ),(0.255128989415, ))
                                ])


# train_data = torchvision.datasets.ImageFolder('./train',transform=transform_train)
# test_data = torchvision.datasets.ImageFolder('./test',transform=transform_test)
train_data = DepressionDataset(csv_file = trainlist,transform=transform_train)
test_data = DepressionDataset(csv_file = testlist,transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data,   batch_size=test_batch_size, shuffle=False, **kwargs)

In [7]:
# di = iter(train_loader)

In [8]:
# # out = di.next()
# out, lbl = di.next()


In [9]:
# print(out)

In [10]:
class average_meter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [11]:
class VGG_Net(nn.Module):
    def __init__(self, model):
        super(VGG_Net, self).__init__()

        self.pre_model = nn.Sequential(*list(model.children())[:-1])
        # self.dropout = nn.Dropout(p=0.8)
        self.classifier = nn.Linear(4096, 2)

    def forward(self, x):
        x = self.pre_model(x)
        # x = self.dropout(x)
        x = self.classifier(x)

        return x 

In [12]:
model_emotion = VGG_Face_torch
model_emotion.load_state_dict(torch.load('VGG_Face_torch.pth'))
model = VGG_Net(model_emotion).cuda()

# model
# summary(model, input_size=(3, 224, 224))

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum ,weight_decay= 0.0005,nesterov=True)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=20, gamma=0.1)


In [13]:
# model

In [14]:
def train(epoch):
    
    losses = average_meter()
    accuracy = average_meter()
    model.train()

    for batch_idx,sample in enumerate(train_loader):
        data, target = Variable(sample['image']).cuda(), Variable(sample['label']).cuda()       
        output = model(data)  
        loss = loss_function(output, target)
        losses.update(loss.data, data.size(0))
        pred = output.data.max(1)[1]
#         print(pred)
        prec = pred.eq(target.data).cpu().sum()
        accuracy.update(float(prec) / data.size(0), data.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {}\t'
                  'Batch: [{:5d}/{:5d} ({:3.0f}%)]\t'                     
                  'Loss: {:.6f}'.format(
                      epoch, batch_idx * len(data), len(train_data),
                      100. * batch_idx / len(train_loader), losses.val))
            print('Training accuracy:', accuracy.val )

In [15]:
def val():
    losses = average_meter()
    accuracy = average_meter()

    model.eval()
    
    for sample in test_loader:
        with torch.no_grad():
            data, target = Variable(sample['image']).cuda(), Variable(sample['label']).cuda()
        output = model(data)
        loss = loss_function(output, target)
        losses.update(loss.data, data.size(0))
        pred = output.data.max(1)[1]
        prec = pred.eq(target.data).cpu().sum()
        accuracy.update(float(prec) / data.size(0), data.size(0))

    print('\nTest: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        losses.avg, int(accuracy.sum), len(test_data), 100. * accuracy.avg))

    return accuracy.avg

In [16]:
def main():

    best_model = model
    best_accuray = 0.0

    for epoch in range(1, epochs + 1):
        train(epoch)
        val_accuracy = val()
        scheduler.step()


#         if best_accuray < val_accuracy:
#             best_model   = model
#             best_accuray = val_accuracy


#     print ("The best model has an accuracy of " + str(best_accuray))
#     torch.save(best_model.state_dict(), 'best.model')


if __name__ == '__main__':
    main()

Train Epoch: 1	Batch: [    0/71964 (  0%)]	Loss: 0.702595
Training accuracy: 0.3125
Train Epoch: 1	Batch: [  480/71964 (  1%)]	Loss: 0.668723
Training accuracy: 0.8125
Train Epoch: 1	Batch: [  960/71964 (  1%)]	Loss: 0.577329
Training accuracy: 0.6875
Train Epoch: 1	Batch: [ 1440/71964 (  2%)]	Loss: 0.255241
Training accuracy: 0.875
Train Epoch: 1	Batch: [ 1920/71964 (  3%)]	Loss: 0.069277
Training accuracy: 1.0
Train Epoch: 1	Batch: [ 2400/71964 (  3%)]	Loss: 0.006693
Training accuracy: 1.0
Train Epoch: 1	Batch: [ 2880/71964 (  4%)]	Loss: 0.036734
Training accuracy: 1.0
Train Epoch: 1	Batch: [ 3360/71964 (  5%)]	Loss: 0.063268
Training accuracy: 0.9375
Train Epoch: 1	Batch: [ 3840/71964 (  5%)]	Loss: 0.065992
Training accuracy: 1.0
Train Epoch: 1	Batch: [ 4320/71964 (  6%)]	Loss: 0.000826
Training accuracy: 1.0
Train Epoch: 1	Batch: [ 4800/71964 (  7%)]	Loss: 0.002192
Training accuracy: 1.0
Train Epoch: 1	Batch: [ 5280/71964 (  7%)]	Loss: 0.322251
Training accuracy: 0.9375
Train Epoch

Train Epoch: 1	Batch: [48480/71964 ( 67%)]	Loss: 0.000700
Training accuracy: 1.0
Train Epoch: 1	Batch: [48960/71964 ( 68%)]	Loss: 0.000245
Training accuracy: 1.0
Train Epoch: 1	Batch: [49440/71964 ( 69%)]	Loss: 0.079580
Training accuracy: 0.9375
Train Epoch: 1	Batch: [49920/71964 ( 69%)]	Loss: 0.032338
Training accuracy: 1.0
Train Epoch: 1	Batch: [50400/71964 ( 70%)]	Loss: 0.000826
Training accuracy: 1.0
Train Epoch: 1	Batch: [50880/71964 ( 71%)]	Loss: 0.000428
Training accuracy: 1.0
Train Epoch: 1	Batch: [51360/71964 ( 71%)]	Loss: 0.000482
Training accuracy: 1.0
Train Epoch: 1	Batch: [51840/71964 ( 72%)]	Loss: 0.001945
Training accuracy: 1.0
Train Epoch: 1	Batch: [52320/71964 ( 73%)]	Loss: 0.003693
Training accuracy: 1.0
Train Epoch: 1	Batch: [52800/71964 ( 73%)]	Loss: 0.000035
Training accuracy: 1.0
Train Epoch: 1	Batch: [53280/71964 ( 74%)]	Loss: 0.002195
Training accuracy: 1.0
Train Epoch: 1	Batch: [53760/71964 ( 75%)]	Loss: 0.005866
Training accuracy: 1.0
Train Epoch: 1	Batch: [54

Train Epoch: 2	Batch: [24960/71964 ( 35%)]	Loss: 0.000044
Training accuracy: 1.0
Train Epoch: 2	Batch: [25440/71964 ( 35%)]	Loss: 0.000040
Training accuracy: 1.0
Train Epoch: 2	Batch: [25920/71964 ( 36%)]	Loss: 0.000064
Training accuracy: 1.0
Train Epoch: 2	Batch: [26400/71964 ( 37%)]	Loss: 0.000057
Training accuracy: 1.0
Train Epoch: 2	Batch: [26880/71964 ( 37%)]	Loss: 0.000041
Training accuracy: 1.0
Train Epoch: 2	Batch: [27360/71964 ( 38%)]	Loss: 0.000141
Training accuracy: 1.0
Train Epoch: 2	Batch: [27840/71964 ( 39%)]	Loss: 0.000004
Training accuracy: 1.0
Train Epoch: 2	Batch: [28320/71964 ( 39%)]	Loss: 0.000188
Training accuracy: 1.0
Train Epoch: 2	Batch: [28800/71964 ( 40%)]	Loss: 0.000009
Training accuracy: 1.0
Train Epoch: 2	Batch: [29280/71964 ( 41%)]	Loss: 0.001292
Training accuracy: 1.0
Train Epoch: 2	Batch: [29760/71964 ( 41%)]	Loss: 0.000145
Training accuracy: 1.0
Train Epoch: 2	Batch: [30240/71964 ( 42%)]	Loss: 0.000052
Training accuracy: 1.0
Train Epoch: 2	Batch: [30720

Train Epoch: 3	Batch: [ 1440/71964 (  2%)]	Loss: 0.006046
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 1920/71964 (  3%)]	Loss: 0.000195
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 2400/71964 (  3%)]	Loss: 0.012080
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 2880/71964 (  4%)]	Loss: 0.000070
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 3360/71964 (  5%)]	Loss: 0.000095
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 3840/71964 (  5%)]	Loss: 0.000142
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 4320/71964 (  6%)]	Loss: 0.000004
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 4800/71964 (  7%)]	Loss: 0.000008
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 5280/71964 (  7%)]	Loss: 0.000009
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 5760/71964 (  8%)]	Loss: 0.000013
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 6240/71964 (  9%)]	Loss: 0.000022
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 6720/71964 (  9%)]	Loss: 0.000038
Training accuracy: 1.0
Train Epoch: 3	Batch: [ 7200

Train Epoch: 3	Batch: [50400/71964 ( 70%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 3	Batch: [50880/71964 ( 71%)]	Loss: 0.000006
Training accuracy: 1.0
Train Epoch: 3	Batch: [51360/71964 ( 71%)]	Loss: 0.000015
Training accuracy: 1.0
Train Epoch: 3	Batch: [51840/71964 ( 72%)]	Loss: 0.000011
Training accuracy: 1.0
Train Epoch: 3	Batch: [52320/71964 ( 73%)]	Loss: 0.004401
Training accuracy: 1.0
Train Epoch: 3	Batch: [52800/71964 ( 73%)]	Loss: 0.000065
Training accuracy: 1.0
Train Epoch: 3	Batch: [53280/71964 ( 74%)]	Loss: 0.000176
Training accuracy: 1.0
Train Epoch: 3	Batch: [53760/71964 ( 75%)]	Loss: 0.000020
Training accuracy: 1.0
Train Epoch: 3	Batch: [54240/71964 ( 75%)]	Loss: 0.000128
Training accuracy: 1.0
Train Epoch: 3	Batch: [54720/71964 ( 76%)]	Loss: 0.000003
Training accuracy: 1.0
Train Epoch: 3	Batch: [55200/71964 ( 77%)]	Loss: 0.000008
Training accuracy: 1.0
Train Epoch: 3	Batch: [55680/71964 ( 77%)]	Loss: 0.000041
Training accuracy: 1.0
Train Epoch: 3	Batch: [56160

...

Train Epoch: 97	Batch: [56160/71964 ( 78%)]	Loss: 0.000041
Training accuracy: 1.0
Train Epoch: 97	Batch: [56640/71964 ( 79%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 97	Batch: [57120/71964 ( 79%)]	Loss: 0.000155
Training accuracy: 1.0
Train Epoch: 97	Batch: [57600/71964 ( 80%)]	Loss: 0.000061
Training accuracy: 1.0
Train Epoch: 97	Batch: [58080/71964 ( 81%)]	Loss: 0.000004
Training accuracy: 1.0
Train Epoch: 97	Batch: [58560/71964 ( 81%)]	Loss: 0.000002
Training accuracy: 1.0
Train Epoch: 97	Batch: [59040/71964 ( 82%)]	Loss: 0.000004
Training accuracy: 1.0
Train Epoch: 97	Batch: [59520/71964 ( 83%)]	Loss: 0.000545
Training accuracy: 1.0
Train Epoch: 97	Batch: [60000/71964 ( 83%)]	Loss: 0.000000
Training accuracy: 1.0
Train Epoch: 97	Batch: [60480/71964 ( 84%)]	Loss: 0.000008
Training accuracy: 1.0
Train Epoch: 97	Batch: [60960/71964 ( 85%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 97	Batch: [61440/71964 ( 85%)]	Loss: 0.000005
Training accuracy: 1.0
Train Epoch: 97	

Train Epoch: 98	Batch: [32160/71964 ( 45%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 98	Batch: [32640/71964 ( 45%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 98	Batch: [33120/71964 ( 46%)]	Loss: 0.000002
Training accuracy: 1.0
Train Epoch: 98	Batch: [33600/71964 ( 47%)]	Loss: 0.000004
Training accuracy: 1.0
Train Epoch: 98	Batch: [34080/71964 ( 47%)]	Loss: 0.000002
Training accuracy: 1.0
Train Epoch: 98	Batch: [34560/71964 ( 48%)]	Loss: 0.000010
Training accuracy: 1.0
Train Epoch: 98	Batch: [35040/71964 ( 49%)]	Loss: 0.000005
Training accuracy: 1.0
Train Epoch: 98	Batch: [35520/71964 ( 49%)]	Loss: 0.000007
Training accuracy: 1.0
Train Epoch: 98	Batch: [36000/71964 ( 50%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 98	Batch: [36480/71964 ( 51%)]	Loss: 0.000004
Training accuracy: 1.0
Train Epoch: 98	Batch: [36960/71964 ( 51%)]	Loss: 0.000005
Training accuracy: 1.0
Train Epoch: 98	Batch: [37440/71964 ( 52%)]	Loss: 0.000002
Training accuracy: 1.0
Train Epoch: 98	

Train Epoch: 99	Batch: [ 8160/71964 ( 11%)]	Loss: 0.000003
Training accuracy: 1.0
Train Epoch: 99	Batch: [ 8640/71964 ( 12%)]	Loss: 0.000002
Training accuracy: 1.0
Train Epoch: 99	Batch: [ 9120/71964 ( 13%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [ 9600/71964 ( 13%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [10080/71964 ( 14%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [10560/71964 ( 15%)]	Loss: 0.000002
Training accuracy: 1.0
Train Epoch: 99	Batch: [11040/71964 ( 15%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [11520/71964 ( 16%)]	Loss: 0.000030
Training accuracy: 1.0
Train Epoch: 99	Batch: [12000/71964 ( 17%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [12480/71964 ( 17%)]	Loss: 0.000003
Training accuracy: 1.0
Train Epoch: 99	Batch: [12960/71964 ( 18%)]	Loss: 0.000008
Training accuracy: 1.0
Train Epoch: 99	Batch: [13440/71964 ( 19%)]	Loss: 0.000003
Training accuracy: 1.0
Train Epoch: 99	

Train Epoch: 99	Batch: [56160/71964 ( 78%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [56640/71964 ( 79%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [57120/71964 ( 79%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [57600/71964 ( 80%)]	Loss: 0.000003
Training accuracy: 1.0
Train Epoch: 99	Batch: [58080/71964 ( 81%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [58560/71964 ( 81%)]	Loss: 0.000002
Training accuracy: 1.0
Train Epoch: 99	Batch: [59040/71964 ( 82%)]	Loss: 0.000000
Training accuracy: 1.0
Train Epoch: 99	Batch: [59520/71964 ( 83%)]	Loss: 0.000004
Training accuracy: 1.0
Train Epoch: 99	Batch: [60000/71964 ( 83%)]	Loss: 0.000015
Training accuracy: 1.0
Train Epoch: 99	Batch: [60480/71964 ( 84%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [60960/71964 ( 85%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 99	Batch: [61440/71964 ( 85%)]	Loss: 0.000006
Training accuracy: 1.0
Train Epoch: 99	

Train Epoch: 100	Batch: [31680/71964 ( 44%)]	Loss: 0.000031
Training accuracy: 1.0
Train Epoch: 100	Batch: [32160/71964 ( 45%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 100	Batch: [32640/71964 ( 45%)]	Loss: 0.000003
Training accuracy: 1.0
Train Epoch: 100	Batch: [33120/71964 ( 46%)]	Loss: 0.000272
Training accuracy: 1.0
Train Epoch: 100	Batch: [33600/71964 ( 47%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 100	Batch: [34080/71964 ( 47%)]	Loss: 0.000002
Training accuracy: 1.0
Train Epoch: 100	Batch: [34560/71964 ( 48%)]	Loss: 0.000003
Training accuracy: 1.0
Train Epoch: 100	Batch: [35040/71964 ( 49%)]	Loss: 0.000059
Training accuracy: 1.0
Train Epoch: 100	Batch: [35520/71964 ( 49%)]	Loss: 0.000003
Training accuracy: 1.0
Train Epoch: 100	Batch: [36000/71964 ( 50%)]	Loss: 0.000005
Training accuracy: 1.0
Train Epoch: 100	Batch: [36480/71964 ( 51%)]	Loss: 0.000001
Training accuracy: 1.0
Train Epoch: 100	Batch: [36960/71964 ( 51%)]	Loss: 0.000001
Training accuracy: 1.0
Trai

In [17]:
torch.save(model.state_dict(), 'first.model')